In [14]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time
from tqdm import tqdm

In [15]:
# Load environment and setup
load_dotenv()
COINAPI_KEY = os.getenv('COIN_API_KEY')

# Symbols for both ETH and BTC
ETH_SYMBOL_ID = "COINBASE_SPOT_ETH_USD"
BTC_SYMBOL_ID = "COINBASE_SPOT_BTC_USD"

headers = {
    'Accept': 'application/json',
    'X-CoinAPI-Key': COINAPI_KEY
}

In [16]:
# Load existing CSV
df = pd.read_csv('./5s_data/eth_orderbook_coinbase_5s.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'])

print(f"Loaded {len(df)} rows")
print(f"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"Columns: {list(df.columns[:10])}...")

Loaded 241600 rows
Date range: 2025-12-01 02:57:15.727819 to 2025-12-16 15:46:13.244733
Columns: ['timestamp', 'bid_price_1', 'bid_vol_1', 'ask_price_1', 'ask_vol_1', 'bid_price_2', 'bid_vol_2', 'ask_price_2', 'ask_vol_2', 'bid_price_3']...


In [17]:
def get_ohlcv_data(symbol_id, start_time, end_time, period='5SEC'):
    """
    Fetch OHLCV (candlestick) data which includes volume.
    """
    url = f"https://rest.coinapi.io/v1/ohlcv/{symbol_id}/history"
    
    # Format timestamps properly (remove microseconds)
    start_str = start_time.replace(microsecond=0).isoformat() + 'Z'
    end_str = end_time.replace(microsecond=0).isoformat() + 'Z'
    
    params = {
        'period_id': period,
        'time_start': start_str,
        'time_end': end_str,
        'limit': 10000
    }
    
    print(f"  Requesting: {url}")
    print(f"  Period: {period}, Start: {start_str}, End: {end_str}")
    
    for attempt in range(3):
        try:
            r = requests.get(url, headers=headers, params=params)
            print(f"  Status: {r.status_code}")
            
            if r.status_code == 400:
                print(f"  Response: {r.text[:200]}")
                return []
            elif r.status_code == 429:
                print("  Rate limit, waiting...")
                time.sleep(60)
                continue
            elif r.status_code == 500:
                time.sleep(2 ** attempt)
                continue
            
            r.raise_for_status()
            return r.json()
        except requests.exceptions.RequestException as e:
            print(f"  Error: {e}")
            time.sleep(2 ** attempt)
    return []

In [18]:
# Fetch OHLCV data in chunks for the full date range
start_time = df['timestamp'].min()
end_time = df['timestamp'].max()
chunk_size = timedelta(hours=24)  # Fetch 24 hours at a time

print(f"Fetching OHLCV data from {start_time} to {end_time}")
print(f"Date range: {(end_time - start_time).days} days")

# Fetch ETH data
print(f"\n{'='*60}")
print(f"FETCHING ETH DATA")
print(f"{'='*60}")
all_eth_ohlcv = []
current_start = start_time

while current_start < end_time:
    current_end = min(current_start + chunk_size, end_time)
    print(f"\nFetching ETH chunk: {current_start} to {current_end}")
    
    chunk_ohlcv = get_ohlcv_data(ETH_SYMBOL_ID, current_start, current_end, period='5SEC')
    if chunk_ohlcv:
        all_eth_ohlcv.extend(chunk_ohlcv)
        print(f"  Got {len(chunk_ohlcv)} candles")
    else:
        print(f"  No data returned")
    
    current_start = current_end
    time.sleep(1)  # Rate limiting

eth_ohlcv_df = pd.DataFrame(all_eth_ohlcv)
print(f"\nTotal ETH fetched: {len(eth_ohlcv_df)} candles")

# Fetch BTC data
print(f"\n{'='*60}")
print(f"FETCHING BTC DATA")
print(f"{'='*60}")
all_btc_ohlcv = []
current_start = start_time

while current_start < end_time:
    current_end = min(current_start + chunk_size, end_time)
    print(f"\nFetching BTC chunk: {current_start} to {current_end}")
    
    chunk_ohlcv = get_ohlcv_data(BTC_SYMBOL_ID, current_start, current_end, period='5SEC')
    if chunk_ohlcv:
        all_btc_ohlcv.extend(chunk_ohlcv)
        print(f"  Got {len(chunk_ohlcv)} candles")
    else:
        print(f"  No data returned")
    
    current_start = current_end
    time.sleep(1)  # Rate limiting

btc_ohlcv_df = pd.DataFrame(all_btc_ohlcv)
print(f"\nTotal BTC fetched: {len(btc_ohlcv_df)} candles")

# Display samples
if len(eth_ohlcv_df) > 0:
    print(f"\nETH Sample data:")
    print(eth_ohlcv_df.head())
    
if len(btc_ohlcv_df) > 0:
    print(f"\nBTC Sample data:")
    print(btc_ohlcv_df.head())

Fetching OHLCV data from 2025-12-01 02:57:15.727819 to 2025-12-16 15:46:13.244733
Date range: 15 days

FETCHING ETH DATA

Fetching ETH chunk: 2025-12-01 02:57:15.727819 to 2025-12-02 02:57:15.727819
  Requesting: https://rest.coinapi.io/v1/ohlcv/COINBASE_SPOT_ETH_USD/history
  Period: 5SEC, Start: 2025-12-01T02:57:15Z, End: 2025-12-02T02:57:15Z
  Status: 200
  Got 9936 candles
  Status: 200
  Got 9936 candles

Fetching ETH chunk: 2025-12-02 02:57:15.727819 to 2025-12-03 02:57:15.727819
  Requesting: https://rest.coinapi.io/v1/ohlcv/COINBASE_SPOT_ETH_USD/history
  Period: 5SEC, Start: 2025-12-02T02:57:15Z, End: 2025-12-03T02:57:15Z

Fetching ETH chunk: 2025-12-02 02:57:15.727819 to 2025-12-03 02:57:15.727819
  Requesting: https://rest.coinapi.io/v1/ohlcv/COINBASE_SPOT_ETH_USD/history
  Period: 5SEC, Start: 2025-12-02T02:57:15Z, End: 2025-12-03T02:57:15Z
  Status: 200
  Got 9922 candles
  Status: 200
  Got 9922 candles

Fetching ETH chunk: 2025-12-03 02:57:15.727819 to 2025-12-04 02:57:1

In [19]:
# Debug: Check what columns we actually got
if len(eth_ohlcv_df) > 0:
    print("\nETH - Actual columns in API response:")
    print(eth_ohlcv_df.columns.tolist())
    print("\nETH - First row sample:")
    print(eth_ohlcv_df.iloc[0].to_dict())
    
if len(btc_ohlcv_df) > 0:
    print("\nBTC - Actual columns in API response:")
    print(btc_ohlcv_df.columns.tolist())
    print("\nBTC - First row sample:")
    print(btc_ohlcv_df.iloc[0].to_dict())


ETH - Actual columns in API response:
['time_period_start', 'time_period_end', 'time_open', 'time_close', 'price_open', 'price_high', 'price_low', 'price_close', 'volume_traded', 'trades_count']

ETH - First row sample:
{'time_period_start': '2025-12-01T02:57:15.0000000Z', 'time_period_end': '2025-12-01T02:57:20.0000000Z', 'time_open': '2025-12-01T02:57:15.0116240Z', 'time_close': '2025-12-01T02:57:19.9999260Z', 'price_open': 2841.13, 'price_high': 2841.17, 'price_low': 2841.09, 'price_close': 2841.165, 'volume_traded': 0.59791517, 'trades_count': 15}

BTC - Actual columns in API response:
['time_period_start', 'time_period_end', 'time_open', 'time_close', 'price_open', 'price_high', 'price_low', 'price_close', 'volume_traded', 'trades_count']

BTC - First row sample:
{'time_period_start': '2025-12-01T02:57:15.0000000Z', 'time_period_end': '2025-12-01T02:57:20.0000000Z', 'time_open': '2025-12-01T02:57:15.2620260Z', 'time_close': '2025-12-01T02:57:19.7213570Z', 'price_open': 86730.06, 

In [27]:
# Process ETH OHLCV data
print(f"\n{'='*60}")
print(f"MERGING ETH DATA")
print(f"{'='*60}")

# Drop existing columns if they exist (from previous runs)
for col in ['eth_price', 'eth_volume']:
    if col in df.columns:
        df = df.drop(columns=[col])
        print(f"Dropped existing column: {col}")

if len(eth_ohlcv_df) > 0:
    print(f"ETH columns before rename: {eth_ohlcv_df.columns.tolist()}")
    
    # Map column names
    eth_ohlcv_df = eth_ohlcv_df.rename(columns={
        'time_period_start': 'timestamp',
        'price_close': 'eth_price',
        'volume_traded': 'eth_volume'
    })
    
    print(f"ETH columns after rename: {eth_ohlcv_df.columns.tolist()}")
    
    # Check if rename worked
    if 'eth_price' not in eth_ohlcv_df.columns or 'eth_volume' not in eth_ohlcv_df.columns:
        print("ERROR: Column rename failed! Using fallback.")
        df['eth_price'] = (df['bid_price_1'] + df['ask_price_1']) / 2.0
        df['eth_volume'] = 0.0
    else:
        # Convert timezone-aware timestamps to naive
        eth_ohlcv_df['timestamp'] = pd.to_datetime(eth_ohlcv_df['timestamp']).dt.tz_localize(None)
        
        # Merge ETH data
        df = pd.merge_asof(
            df.sort_values('timestamp'),
            eth_ohlcv_df[['timestamp', 'eth_price', 'eth_volume']].sort_values('timestamp'),
            on='timestamp',
            direction='nearest',
            tolerance=pd.Timedelta(seconds=30)
        )
        
        print(f"Merged {len(df)} rows with ETH data")
        print(f"Missing ETH price: {df['eth_price'].isna().sum()} ({(df['eth_price'].isna().sum()/len(df))*100:.2f}%)")
        print(f"Missing ETH volume: {df['eth_volume'].isna().sum()} ({(df['eth_volume'].isna().sum()/len(df))*100:.2f}%)")
else:
    print("\nNo ETH OHLCV data returned - using fallback (mid-price)")
    df['eth_price'] = (df['bid_price_1'] + df['ask_price_1']) / 2.0
    df['eth_volume'] = 0.0

# Process BTC OHLCV data
print(f"\n{'='*60}")
print(f"MERGING BTC DATA")
print(f"{'='*60}")

# Drop existing columns if they exist (from previous runs)
for col in ['btc_price', 'btc_volume']:
    if col in df.columns:
        df = df.drop(columns=[col])
        print(f"Dropped existing column: {col}")

if len(btc_ohlcv_df) > 0:
    print(f"BTC columns before rename: {btc_ohlcv_df.columns.tolist()}")
    
    # Map column names
    btc_ohlcv_df = btc_ohlcv_df.rename(columns={
        'time_period_start': 'timestamp',
        'price_close': 'btc_price',
        'volume_traded': 'btc_volume'
    })
    
    print(f"BTC columns after rename: {btc_ohlcv_df.columns.tolist()}")
    
    # Check if rename worked
    if 'btc_price' not in btc_ohlcv_df.columns or 'btc_volume' not in btc_ohlcv_df.columns:
        print("ERROR: Column rename failed! Setting to NaN.")
        df['btc_price'] = None
        df['btc_volume'] = None
    else:
        # Convert timezone-aware timestamps to naive
        btc_ohlcv_df['timestamp'] = pd.to_datetime(btc_ohlcv_df['timestamp']).dt.tz_localize(None)
        
        # Merge BTC data
        df = pd.merge_asof(
            df.sort_values('timestamp'),
            btc_ohlcv_df[['timestamp', 'btc_price', 'btc_volume']].sort_values('timestamp'),
            on='timestamp',
            direction='nearest',
            tolerance=pd.Timedelta(seconds=30)
        )
        
        print(f"Merged {len(df)} rows with BTC data")
        print(f"Missing BTC price: {df['btc_price'].isna().sum()} ({(df['btc_price'].isna().sum()/len(df))*100:.2f}%)")
        print(f"Missing BTC volume: {df['btc_volume'].isna().sum()} ({(df['btc_volume'].isna().sum()/len(df))*100:.2f}%)")
else:
    print("\nNo BTC OHLCV data returned - setting to NaN")
    df['btc_price'] = None
    df['btc_volume'] = None



MERGING ETH DATA
Dropped existing column: eth_price
Dropped existing column: eth_volume
ETH columns before rename: ['timestamp', 'time_period_end', 'time_open', 'time_close', 'price_open', 'price_high', 'price_low', 'eth_price', 'eth_volume', 'trades_count']
ETH columns after rename: ['timestamp', 'time_period_end', 'time_open', 'time_close', 'price_open', 'price_high', 'price_low', 'eth_price', 'eth_volume', 'trades_count']
Merged 241600 rows with ETH data
Missing ETH price: 98677 (40.84%)
Missing ETH volume: 98677 (40.84%)

MERGING BTC DATA
Dropped existing column: btc_price
Dropped existing column: btc_volume
BTC columns before rename: ['timestamp', 'time_period_end', 'time_open', 'time_close', 'price_open', 'price_high', 'price_low', 'btc_price', 'btc_volume', 'trades_count']
BTC columns after rename: ['timestamp', 'time_period_end', 'time_open', 'time_close', 'price_open', 'price_high', 'price_low', 'btc_price', 'btc_volume', 'trades_count']
Merged 241600 rows with BTC data
Missi

In [ ]:
# Save updated CSV
output_path = './5s_data/eth_orderbook_coinbase_5s_with_price_volume.csv'
df.to_csv(output_path, index=False)
print(f"\nSaved to: {output_path}")
print(f"Final shape: {df.shape}")
print(f"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"Duration: {(df['timestamp'].max() - df['timestamp'].min()).days} days")

# Preview
print("\nPreview:")
preview_cols = ['timestamp', 'bid_price_1', 'ask_price_1', 'eth_price', 'eth_volume', 'btc_price', 'btc_volume']
print(df[preview_cols].head(10))

total_rows = len(df)

eth_price_missing = df['eth_price'].isna().sum()
eth_volume_missing = df['eth_volume'].isna().sum()
btc_price_missing = df['btc_price'].isna().sum()
btc_volume_missing = df['btc_volume'].isna().sum()

# Statistics for available data
print(f"\n{'='*60}")
print(f"DATA STATISTICS (Non-null values only)")
print(f"{'='*60}")

if eth_price_missing < total_rows:
    print(f"\nETH:")
    print(f"  Price range: ${df['eth_price'].min():.2f} - ${df['eth_price'].max():.2f}")
    print(f"  Mean price: ${df['eth_price'].mean():.2f}")
    print(f"  Mean volume: {df['eth_volume'].mean():.6f} ETH")
    print(f"  Total volume: {df['eth_volume'].sum():.2f} ETH")

if btc_price_missing < total_rows:
    print(f"\nBTC:")
    print(f"  Price range: ${df['btc_price'].min():.2f} - ${df['btc_price'].max():.2f}")
    print(f"  Mean price: ${df['btc_price'].mean():.2f}")
    print(f"  Mean volume: {df['btc_volume'].mean():.6f} BTC")
    print(f"  Total volume: {df['btc_volume'].sum():.2f} BTC")


Saved to: ./5s_data/eth_orderbook_coinbase_5s_with_price_volume.csv
Final shape: (241600, 53)
Date range: 2025-12-01 02:57:15.727819 to 2025-12-16 15:46:13.244733
Duration: 15 days

Preview:
                   timestamp  bid_price_1  ask_price_1  eth_price  eth_volume  \
0 2025-12-01 02:57:15.727819      2841.16      2841.17   2841.165    0.597915   
1 2025-12-01 02:57:21.073133      2841.10      2841.11   2841.085    0.777114   
2 2025-12-01 02:57:26.375175      2840.82      2841.00   2840.990    1.177497   
3 2025-12-01 02:57:31.660663      2841.00      2841.01   2841.005    0.025164   
4 2025-12-01 02:57:37.007031      2841.00      2841.01   2841.005    0.003338   
5 2025-12-01 02:57:42.671388      2841.11      2841.12   2842.230    2.630444   
6 2025-12-01 02:57:47.996316      2841.53      2841.54   2841.730    2.091136   
7 2025-12-01 02:57:53.320210      2841.72      2841.73   2841.500    0.349916   
8 2025-12-01 02:57:58.597944      2841.32      2841.33   2841.490    0.043777  